In [37]:
#Import all necessary packages
import pandas as pd
import numpy as np
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import pvlib
import pvanalytics
import pvpltools
import plantpredict
import math
import time
import scipy
import seaborn as sns

#Import all necessary modules from packages
from datetime import datetime
from pvpltools import iec61853
from pvpltools.module_efficiency import adr, heydenreich, motherpv, pvgis, mpm5, mpm6, bilinear
from pvpltools.module_efficiency import fit_efficiency_model, fit_bilinear
from pandas.plotting import register_matplotlib_converters
from plantpredict.enumerations import PredictionStatusEnum, TranspositionModelEnum, SpectralShiftModelEnum,DiffuseDirectDecompositionModelEnum, ModuleTemperatureModelEnum, IncidenceAngleModelTypeEnum, AirMassModelTypeEnum, DirectBeamShadingModelEnum, SoilingModelTypeEnum, DegradationModelEnum, TrackingTypeEnum, BacktrackingTypeEnum, DiffuseShadingModelEnum
register_matplotlib_converters()
from scipy.optimize import minimize

#Set style / plotting preferences
mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
mpl.rcParams['axes.titlesize']=22
sns.set(rc={'figure.figsize':(12,7)})
sns.set_style('whitegrid')

In [38]:
api = plantpredict.Api(
    username="lelia.deville1@louisiana.edu.plantpredictapi",
    password="r9HBf70PC6dnr/kp3dwc9eBIzYQWMMP+7ekjM+EIXU2jl5pJf+AQ9/ZRVz6AAJUrad6vZ+KWZaoeTwsZHg65Sg==",
    client_id="0oadalz1takAapQQx357",
    client_secret="mEoW-QTKXPmzLPavJfMCymXD9gsqjQ9-bHhCoFJg")

In [39]:
#Define Project Name and Latitude/Longitude & Create
project = api.project(name="Powerplant A", latitude=30.22,longitude=-92.02)
project.assign_location_attributes()
project.create()

{'id': 76406}

In [40]:
prediction = api.prediction(project_id=project.id, name="Prediction A")
prediction.weather_id = 13628
weather = api.weather(id=prediction.weather_id)
weather.get()
prediction.start_date = weather.start_date
prediction.end_date = weather.end_date
prediction.start = weather.start_date
prediction.end = weather.end_date

In [41]:
from plantpredict.enumerations import PredictionStatusEnum, TranspositionModelEnum, SpectralShiftModelEnum, \
    DiffuseDirectDecompositionModelEnum, ModuleTemperatureModelEnum, IncidenceAngleModelTypeEnum, \
    AirMassModelTypeEnum, DirectBeamShadingModelEnum, SoilingModelTypeEnum, DegradationModelEnum, \
    TrackingTypeEnum, BacktrackingTypeEnum, DiffuseShadingModelEnum

prediction.diffuse_direct_decomp_model = DiffuseDirectDecompositionModelEnum.NONE
prediction.transposition_model = TranspositionModelEnum.PEREZ
prediction.mod_temp_model = ModuleTemperatureModelEnum.HEAT_BALANCE
prediction.inc_angle_model = IncidenceAngleModelTypeEnum.TABULAR_IAM
prediction.spectral_shift_model = SpectralShiftModelEnum.TWO_PARAM_PWAT_AND_AM
prediction.air_mass_model = AirMassModelTypeEnum.BIRD_HULSTROM
prediction.direct_beam_shading_model = DirectBeamShadingModelEnum.LINEAR
prediction.diffuse_shading_model = DiffuseShadingModelEnum.SCHAAR_PANCHULA
prediction.soiling_model = SoilingModelTypeEnum.CONSTANT_MONTHLY
prediction.monthly_factors = [
    {"month": 1, "month_name": "Jan", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 2, "month_name": "Feb", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 3, "month_name": "Mar", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 4, "month_name": "Apr", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 5, "month_name": "May", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 6, "month_name": "Jun", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 7, "month_name": "Jul", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 8, "month_name": "Aug", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 9, "month_name": "Sep", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 10, "month_name": "Oct", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 11, "month_name": "Nov", "albedo": 0.2, "soiling_loss": 2.0},
    {"month": 12, "month_name": "Dec", "albedo": 0.2, "soiling_loss": 2.0},
]
prediction.diffuse_direct_decomp_model_executed = True
prediction.use_meteo_dni = False
prediction.use_meteo_poai = False
prediction.degradation_model = DegradationModelEnum.LINEAR_DC
prediction.linear_degradation_rate = 0.5
prediction.first_year_degradation = False
prediction.year_repeater = 3

In [42]:
prediction.create()

{'id': 425992}

In [43]:
powerplant = api.powerplant(project_id=project.id, prediction_id=prediction.id)

In [44]:
fixed_tilt_block_name = powerplant.add_block()
fixed_tilt_array_name = powerplant.add_array(
    block_name=fixed_tilt_block_name,
    transformer_enabled=False,
    repeater=3,
    description="Arrays in north eastern section of plant."
)
fixed_tilt_inverter_name = powerplant.add_inverter(
    block_name=fixed_tilt_block_name,
    array_name=fixed_tilt_array_name,
    inverter_id=619,
    setpoint_kw=720.0,
    repeater=2
)

In [45]:
field_dc_power = powerplant.calculate_field_dc_power_from_dc_ac_ratio(dc_ac_ratio=1.2, inverter_setpoint=720.0)
post_to_post_spacing = powerplant.calculate_post_to_post_spacing_from_gcr(ground_coverage_ratio=0.40, module_id=298,
                                                                          modules_high=4)

fixed_tilt_dc_field_name = powerplant.add_dc_field(
    block_name=fixed_tilt_block_name,
    array_name=fixed_tilt_array_name,
    inverter_name=fixed_tilt_inverter_name,
    module_id=298,
    tracking_type=TrackingTypeEnum.FIXED_TILT,
    modules_high=4,
    modules_wired_in_series=10,
    post_to_post_spacing=post_to_post_spacing,
    number_of_rows=10,
    field_dc_power=field_dc_power,
    module_tilt=30
)

In [46]:
powerplant.create()

{'is_successful': True}

In [63]:
powerplant.update()

{'changes_made': True}

In [47]:
powerplant.get()

{'id': 577642,
 'result_list': None,
 'blocks': [{'name': 1,
   'nodal_export_options': None,
   'result_list': None,
   'arrays': [{'name': 1,
     'result_list': None,
     'inverters': [{'name': 'A',
       'result_list': None,
       'dc_fields': [{'name': 1,
         'module_id': 298,
         'module': {'status': 6,
          'model': 'FS-4115-3 Feb 2016',
          'manufacturer': 'First Solar',
          'length': 1200.0,
          'width': 600.0,
          'weight': 12.0,
          'default_orientation': 0,
          'number_of_cells_in_series': 108,
          'number_of_cells_in_parallel': 2,
          'stc_short_circuit_current': 1.83,
          'stc_open_circuit_voltage': 87.6,
          'stc_mpp_current': 1.66,
          'stc_mpp_voltage': 69.3,
          'stc_max_power': 115.0,
          'stc_power_temp_coef': -0.28,
          'stc_short_circuit_current_temp_coef': 0.038,
          'stc_open_circuit_voltage_temp_coef': -0.28,
          'stc_efficiency': 15.97,
          '

In [48]:
prediction.run()

{'is_successful': True}

In [49]:
prediction.get_results_summary()

{'id': 626440,
 'prediction_name': 'Area 51 - Contracted',
 'run_by': 'Lelia Deville`',
 'time_of_prediction': '2021-11-18T19:27:55.753',
 'elapsed_time_minutes': 0.10975161666666666,
 'exec_date': '2021-11-18T19:27:55.753',
 'tool_name': 'PlantPredict',
 'tool_version': '8.27.0.25492',
 'start': '2005-01-01T00:00:00',
 'end': '2005-12-31T23:00:00',
 'decomposition_model': 'None',
 'transposition_model': 'Perez',
 'inc_model': 'Tabular IAM',
 'air_mass_model': 'Bird-Hulstrom',
 'module_temperature_model': 'Heat Balance',
 'degradation_model': 'Linear DC',
 'direct_shading_model': 'Linear',
 'diffuse_shading_model': 'Schaar-Panchula',
 'degradation_rate': 0.005,
 'non_linear_degradation_rates': None,
 'light_and_elevation_temperature_degradation_rates': None,
 'spectral_shift_model': '2-Param Pwat and AM',
 'use_weather_dni': False,
 'use_meteo_poai': False,
 'soiling_model': 'Constant Monthly',
 'error_irr_acc': 0.06480740698407861,
 'spatial_variability': 0.02,
 'interannual_variabili